In [67]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
import statsmodels.api as sm
from scipy import stats
from sklearn.preprocessing import StandardScaler


In [68]:
data = pd.read_csv('active_user.csv', header = 0 )
#standardize the x 
df = StandardScaler().fit_transform(data[['Users', 'Pageviews']])


In [94]:
test=data.iloc[0:12, 1:]
test.corr()['Active Users']

Users           0.603282
Pageviews       0.357676
Active Users    1.000000
Name: Active Users, dtype: float64

In [74]:
'''split x and y into Numpy'''
#x= test[['Users', 'Pageviews']].to_numpy()
x = df[0:12]
y = test['Active Users'].to_numpy()
x, y

(array([[-1.47321101, -1.60767618],
        [-1.35753116, -1.2155475 ],
        [-1.482851  , -1.16892389],
        [-0.95265168, -0.5444276 ],
        [-0.84179183,  0.16632608],
        [-1.14545143,  0.1432082 ],
        [-0.29472254, -0.59307311],
        [-0.10674278, -0.51606693],
        [-1.23462132,  0.20637975],
        [ 0.51985641,  0.64291283],
        [ 0.01375707, -0.24838085],
        [-0.64899208,  1.76735995]]),
 array([23881.63, 26855.11, 24173.67, 23421.84, 24372.17, 25428.86,
        28632.92, 23247.43, 22590.86, 28883.42, 31148.66, 28846.48]))

In [75]:
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     2.696
Date:                Fri, 11 Sep 2020   Prob (F-statistic):              0.121
Time:                        22:49:26   Log-Likelihood:                -109.00
No. Observations:                  12   AIC:                             224.0
Df Residuals:                       9   BIC:                             225.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.783e+04   1126.474     24.707      0.0

C:\Users\linthan\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


In [93]:
x_user= df[0:12,0]
c_X_user = sm.add_constant(x_user)
est_user = sm.OLS(y, c_X_user)
est_user2 = est_user.fit()
print(est_user2.summary())
x_user_test = df[12:,0]
c_X_user_test = sm.add_constant(x_user_test)
y_predict = est_user2.predict(c_X_user_test)
y_predict

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     5.722
Date:                Fri, 11 Sep 2020   Prob (F-statistic):             0.0378
Time:                        23:12:51   Log-Likelihood:                -109.11
No. Observations:                  12   AIC:                             222.2
Df Residuals:                      10   BIC:                             223.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.791e+04   1061.737     26.285      0.0

array([27999.78277608, 27492.33023114, 27824.36708153, 28663.85647686,
       29584.78887323, 29691.29125921, 31082.08712312, 31320.15128   ,
       29246.48717661, 34032.82969924, 31577.00997559, 29785.26395271])

In [86]:
x_page= df[0:12,1]
c_X_page = sm.add_constant(x_page)
est = sm.OLS(y, c_X_page)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.467
Date:                Fri, 11 Sep 2020   Prob (F-statistic):              0.254
Time:                        23:01:12   Log-Likelihood:                -111.00
No. Observations:                  12   AIC:                             226.0
Df Residuals:                      10   BIC:                             227.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.623e+04    826.932     31.717      0.0

In [60]:
#leave one out for LinearRegression , tempting to use users as x only 
loo = LeaveOneOut()
y_true, y_pred = list(), list()
for train_index, test_index in loo.split(x):
    x_train, x_test = x[train_index, :], x[test_index,:]
    #print ('Xtrain:' ,  x_train , "X_Test:", x_test)
    y_train, y_test = y[train_index], y[test_index]
    model= LinearRegression()
    model.fit(x_train, y_train)
    yhat= model.predict(x_test)
    y_true.append(y_test[0])
    y_pred.append(yhat[0])
r2 = metrics.r2_score(y_true, y_pred)
print ('r2:', r2)


r2: 0.10499052559210564


In [22]:
model.predict(data[['Users', 'Pageviews' ]].loc[12:].to_numpy())

array([28957.19337179, 27912.01637274, 28222.44735074, 28679.13714102,
       28873.14398286, 29007.00145799, 31561.03561307, 31659.99868066,
       28407.36012005, 33897.18723846, 31668.62458789, 27421.8251545 ])